In [2]:
%cd c:\pytest

c:\pytest


In [3]:
def read_data(filename, encoding='cp949'): # 읽기 함수 정의
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # txt 파일의 헤더(id document label)는 제외하기
    return data
def write_data(data, filename, encoding='cp949'): # 쓰기 함수도 정의
    with open(filename, 'w', encoding=encoding) as f:
        f.write(data)
data = read_data('ratings.txt', encoding='cp949')

In [4]:
from tqdm import tqdm # 진행 정도 출력
import rhinoMorph
rn = rhinoMorph.startRhino()

morphed_data = ''
for data_each in tqdm(data):
    morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[1],
    pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
    joined_data_each = ' '.join(morphed_data_each) # 문자열을 하나로 연결
    if joined_data_each: # 내용이 있는 경우만 저장하게 함
        morphed_data += data_each[0]+"\t"+joined_data_each+"\t"+data_each[2]+"\n"
# 형태소 분석된 파일 저장
# write_data(morphed_data, 'ratings_morphed.txt', encoding='cp949')

filepath:  c:\projects\venv\lib\site-packages
classpath:  c:\projects\venv\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


100%|██████████| 200000/200000 [00:00<00:00, 4095360.10it/s]


In [6]:
data = read_data('ratings_morphed.txt' , encoding='cp949')
print(len(data))
print(len(data[0]))
print(data[0])

197559
3
['8132799', '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사', '1']


In [9]:
data_id=[line[0] for line in data]
data_text=[line[1] for line in data] 
data_senti=[line[2] for line in data]
positive=read_data('positive.txt')
negative=read_data('negative.txt')
print("positive:", positive[:10])
print("negatvie:", negative[:10])
pos_found=[]
neg_found=[]

positive: [['가능'], ['가능하'], ['가락'], ['가치'], ['간단하'], ['간단히'], ['간직하'], ['간호'], ['간호하'], ['감동']]
negatvie: [['가로막'], ['가로지르'], ['가리'], ['가명'], ['가시'], ['가열'], ['가열되'], ['간섭하'], ['간신히'], ['갇히']]


In [10]:
def cntWordInLine(data, senti):
    senti_found=[]
    for onedata in data:
        oneline_word= onedata.split()
        senti_temp = 0
        for sentiword in senti:
            if sentiword[0] in oneline_word:
                senti_temp+=1
        senti_found.append(senti_temp)
    return senti_found

data_senti_poscnt = cntWordInLine(data_text, positive)
data_senti_negcnt = cntWordInLine(data_text, negative)

print(data_senti_poscnt[:20])
print(data_senti_negcnt[:20])

[5, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0]
[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]


In [17]:
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text, 'original':data_senti, 
'pos':data_senti_poscnt, 'neg':data_senti_negcnt})
senti_score = newdata['pos'] - newdata['neg'] # 긍정개수에서 부정개수를 뺌
newdata['senti_score'] = senti_score

In [20]:
newdata.loc[newdata.senti_score > 0, 'new'] = 1 # 새로운 긍부정 기호
newdata.loc[newdata.senti_score <= 0, 'new'] = 0 # 새로운 긍부정 기호

In [21]:
newdata

,id,text,original,pos,neg,senti_score,new
0,8132799,디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...,1,5,1,4,1.0
1,4655635,폴리스스토리 시리즈 뉴 없 최고,1,1,0,1,1.0
2,9251303,와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화,1,0,1,-1,0.0
3,10067386,안개 자욱 하 밤하늘 뜨 초승달 같 영화,1,0,0,0,0.0
4,2190435,사랑 해보 사람 처음 끝 웃 있 영화,1,2,0,2,1.0
...,...,...,...,...,...,...,...
197554,8963373,포켓 몬스터 짜가 ㅡㅡ,0,0,0,0,0.0
197555,3302770,쓰 레 기,0,0,0,0,0.0
197556,5458175,완전 사이코 영화 마지막 더욱더 이 영화 의 질 떨,0,0,1,-1,0.0
197557,6908648,왜 재미없 ㅠㅠ 라따뚜이 보 스머프 보 그렇 ㅋㅋ,0,0,0,0,0.0


In [30]:
newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True'
newdata.loc[pd.to_numeric(newdata.original) != newdata.new, 'matched'] = 'False'
newdata

,id,text,original,pos,neg,senti_score,new,matched
0,8132799,디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...,1,5,1,4,1.0,True
1,4655635,폴리스스토리 시리즈 뉴 없 최고,1,1,0,1,1.0,True
2,9251303,와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화,1,0,1,-1,0.0,False
3,10067386,안개 자욱 하 밤하늘 뜨 초승달 같 영화,1,0,0,0,0.0,False
4,2190435,사랑 해보 사람 처음 끝 웃 있 영화,1,2,0,2,1.0,True
...,...,...,...,...,...,...,...,...
197554,8963373,포켓 몬스터 짜가 ㅡㅡ,0,0,0,0,0.0,True
197555,3302770,쓰 레 기,0,0,0,0,0.0,True
197556,5458175,완전 사이코 영화 마지막 더욱더 이 영화 의 질 떨,0,0,1,-1,0.0,True
197557,6908648,왜 재미없 ㅠㅠ 라따뚜이 보 스머프 보 그렇 ㅋㅋ,0,0,0,0,0.0,True


In [32]:
score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() + newdata.matched.str.count('False').sum()) * 100
print(score)
newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False) # csv 저장
newdata.to_csv('newfile2.txt', sep='\t', encoding='cp949', index=False) # 또는 txt 저장

62.60711989835948


In [33]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))
newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)

In [34]:
newdata

,id,text,original,pos,neg,senti_score,new,matched,sigmoid
0,8132799,디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...,1,5,1,4,1.0,True,0.982014
1,4655635,폴리스스토리 시리즈 뉴 없 최고,1,1,0,1,1.0,True,0.731059
2,9251303,와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화,1,0,1,-1,0.0,False,0.268941
3,10067386,안개 자욱 하 밤하늘 뜨 초승달 같 영화,1,0,0,0,0.0,False,0.500000
4,2190435,사랑 해보 사람 처음 끝 웃 있 영화,1,2,0,2,1.0,True,0.880797
...,...,...,...,...,...,...,...,...,...
197554,8963373,포켓 몬스터 짜가 ㅡㅡ,0,0,0,0,0.0,True,0.500000
197555,3302770,쓰 레 기,0,0,0,0,0.0,True,0.500000
197556,5458175,완전 사이코 영화 마지막 더욱더 이 영화 의 질 떨,0,0,1,-1,0.0,True,0.268941
197557,6908648,왜 재미없 ㅠㅠ 라따뚜이 보 스머프 보 그렇 ㅋㅋ,0,0,0,0,0.0,True,0.500000


In [35]:
import rhinoMorph
rn = rhinoMorph.startRhino()
txt = input()
rhinoMorph.onlyMorph_list(rn, txt, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])

filepath:  c:\projects\venv\lib\site-packages
classpath:  c:\projects\venv\lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


['부모', '안', '계시', '잘못']

In [59]:
import rhinoMorph
rn = rhinoMorph.startRhino()
def morph(sentence):
    result = rhinoMorph.onlyMorph_list(rn, sentence, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi=True)
    return result

filepath:  c:\projects\venv\lib\site-packages
classpath:  c:\projects\venv\lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


In [58]:
morph("귀찮아")

['귀찮']

In [60]:
import pandas as pd
data = pd.read_csv("C:\\pytest\\감성대화말뭉치\\220920_감성사전_조별과제.csv")
data = data.set_index('num')
data['키워드']= data.사람문장1.apply(morph)

In [61]:
data

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,키워드
num,,,,,,,,,
2801,47362,청소년,여성,학업 및 진로,해당없음,슬픔,비통한,부모님 안 계신 게 내 잘못은 아니잖아.,"[부모, 안, 계시다, 잘못]"
2802,28414,청소년,여성,학업 및 진로,해당없음,상처,질투하는,친구와 성적이 비교되는 내가 너무 초라해지는 것 같아.,"[친구, 성적, 비교, 되다, 너무, 초라, 같다]"
2803,15838,중년,여성,"직장, 업무 스트레스",해당없음,상처,괴로워하는,나이 먹어서 새로운 일을 시작한다는 게 역시 쉬운 일은 아닌가 봐.,"[나이, 새롭다, 일, 시작, 역시, 쉽다, 일, 보다]"
2804,26437,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,나 드디어 이번 회사에는 취직될 것 같아.,"[드디어, 이번, 회사, 취직, 되다, 같다]"
2805,42286,노년,남성,"건강, 죽음",만성질환 무,기쁨,기쁨,윗집이 이사 가서 드디어 층간 소음 없이 밤에 깰 일 없이 잘 수 있게 되었어.,"[윗집, 이사, 가다, 드디어, 층, 소음, 없이, 밤, 깨다, 일, 없이, 잘, ..."
...,...,...,...,...,...,...,...,...,...
2996,8718,청소년,여성,학교폭력/따돌림,해당없음,불안,당혹스러운,친한 친구와 싸웠는데 그 친구가 이상한 소문을 내고 다녀서 화나.,"[한, 친구, 싸우다, 그, 친구, 이상, 하다, 소문, 다니다, 화나다]"
2997,8644,노년,남성,대인관계,만성질환 유,당황,고립된,친구들이 서울 외곽으로 이사를 하고 나니 자주 만날 수 없어서 기분이 안 좋아.,"[친구, 서울, 외곽, 이사, 하다, 자주, 만날, 수, 없다, 기분, 안, 좋다]"
2998,19860,중년,여성,"대인관계(부부, 자녀)",해당없음,불안,조심스러운,요즘 딸아이 때문에 힘이 들어.,"[요즘, 딸아이, 힘, 들다]"


In [62]:
data.to_csv("조별발표과제.csv", encoding='cp949')

In [63]:
aa = pd.read_csv("2400_2599.csv")
aa

,Unnamed: 0,긍정,부정,Unnamed: 3,Unnamed: 4
0,2400,NaN,"참견을, 스트레스",NaN,NaN
1,2401,NaN,희생하면서,NaN,NaN
2,2402,일찍,너무,NaN,NaN
3,2403,NaN,"수술비, 너무, 어려워",NaN,NaN
4,2404,NaN,떨려,NaN,NaN
...,...,...,...,...,...
195,2595,NaN,"미안하고, 속상해",NaN,NaN
196,2596,NaN,쇼핑중독인,NaN,NaN
197,2597,NaN,"너무, 걱정이야",2.0,NaN
198,2598,NaN,NaN,2.0,NaN


In [64]:
aa.to_csv("2400_2599.csv", encoding='cp949')